In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel, TFAutoModelForSequenceClassification
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [2]:
train_data = pd.read_csv('data/train_add_normal.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_data

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
4261,4261,일반 대화,"야, 적 저격수 위치 확인했냐?\n몰라, 어딘가 숨어있어.\n아니 ㅅㅂ 이미 쏘고 ..."
4262,4262,일반 대화,"팀원들 왜 이렇게 못 해??\n몰라, 다들 손가락 부상당했나 봄.\n아니 우리 팀 ..."
4263,4263,일반 대화,야 적 네 뒤에! 뒤돌아봐!\n뭐? 뭐? 어디?\n아니 벌써 죽었냐? ㅋㅋㅋㅋㅋ\n...
4264,4264,일반 대화,"야야 거점 뺏긴다!!\n아니 팀원들 뭐함? 다 자냐?\n혼자 막기 개에바야, 도와줘..."


In [4]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반 대화'],
      dtype=object)

In [5]:
label = {'협박 대화' : 0, '갈취 대화' : 1, '직장 내 괴롭힘 대화' :2, '기타 괴롭힘 대화' : 3, '일반 대화' : 4}

In [6]:
train_data['int_class'] = train_data['class'].map(label)

In [7]:
train_data

,idx,class,conversation,int_class
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1
...,...,...,...,...
4261,4261,일반 대화,"야, 적 저격수 위치 확인했냐?\n몰라, 어딘가 숨어있어.\n아니 ㅅㅂ 이미 쏘고 ...",4
4262,4262,일반 대화,"팀원들 왜 이렇게 못 해??\n몰라, 다들 손가락 부상당했나 봄.\n아니 우리 팀 ...",4
4263,4263,일반 대화,야 적 네 뒤에! 뒤돌아봐!\n뭐? 뭐? 어디?\n아니 벌써 죽었냐? ㅋㅋㅋㅋㅋ\n...,4
4264,4264,일반 대화,"야야 거점 뺏긴다!!\n아니 팀원들 뭐함? 다 자냐?\n혼자 막기 개에바야, 도와줘...",4


In [8]:
from transformers import BertTokenizer, TFBertModel

In [9]:
MODEL_NAME = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = TFBertModel.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [10]:
# 테스트 데이터 토큰화 (문장별 토큰 개수 확인)
tokenized_texts = [tokenizer.tokenize(text) for text in train_data['conversation']]

# 각 문장의 토큰 개수 계산
token_lengths = [len(tokens) for tokens in tokenized_texts]

# 최대 토큰 개수 찾기
max_token_length = max(token_lengths)

print(f"문장에서 가장 긴 토큰 개수: {max_token_length}")

문장에서 가장 긴 토큰 개수: 386


In [11]:
# 단어 사전 개수 확인
tokenizer.vocab_size

30000

In [12]:
# 특수 토큰 확인
print(tokenizer.cls_token)
print(tokenizer.sep_token)

[CLS]
[SEP]


In [13]:
# 최대 토큰 인텍스, 문장, 토큰 수 확인
max_token_idx = token_lengths.index(max(token_lengths))  # 가장 긴 문장의 인덱스
max_token_text = train_data['conversation'].iloc[max_token_idx]  # 해당 문장
max_token_count = token_lengths[max_token_idx]

In [14]:
max_token_text

'이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 딱 봐도 많이 좋아지셨잖아. 굳이 수술까진 할 필요 없어보여서.\n 뭐? 그걸 왜 누나가 정해? 누나가 의사야? 병원가서 정밀검사 받아보고 해야지. 큰 병이면 어쩌려고 그래?\n 아 늙어서 몸도 성하지 않는 사람 수술해서 뭐 어쩔건데????\n 그럼 내가 준 수술비는 어쨌는데. 급하다며 급하다고 해서 내가 돈 다 빼서 줬잖아. 그거 어딨냐고 !!!!!!!!!\n 그건 다른 급한 곳에 썼어.\n 하. 급한 곳? 뭔데 어디다 썼는데.\n 엄마 모시고 살면 돈 나가는 곳이 한 두 군데니? 이것 저것 급한 곳들 돈 썼지. 니가 말하면 알아?\n 이건 뭐야? 못 보던건데. 누나 또 명품샀어?\n 아 내놔!!!!!!!! 니가 뭘 알아. 아니야 이거 원래 있던거야.\n 하. 니가 그러고도 인간이야? 그게 어떤돈인데!!!!!!!!!!!! 내가 밤낮으로 잠도 못자가면서 번 돈 !!!!!! 엄마 수술비하라고 다 빼서 준 걸 니가 가방을 쳐 사는데 써???????????\n 아 시끄러워. 나 바빠 가야해.\n 이성을 잃은 듯 너같은 년이 죽어야하는데. 너같은 년은 살 가치가 없다. 너가 죽어 너가 !!!!!!!!!!!\n 왜.왜이래??! 미쳤어?\n 이제 도저히 못참겠다. 몇 년 동안 이게 몇 번째야? 이 짓거리 못하도록 그냥 널 병신 만드는게 낫겠다. 팔 다리를 부러뜨려야 밖으로 안싸돌아 다니지? 발목을 아주 잘라버려야겠어. 주변에 흉기를 찾는다\n 미.미쳤어 왜이렇게 흥분해?!?!?\n 공구함에 있던 망치를 들며 너 내가 오늘 가만 안둬. 미친 건 너야. 팔다리를 없애버리겠어.\n 꺄아아아'

In [15]:
max_token_count

386

In [16]:
MAX_LEN = max_token_count + 2 # 문장 최대 토큰 길이
NUM_LABELS = 5  # 분류할 클래스 개수
BATCH_SIZE = 8
EPOCHS = 5

In [17]:
# 텍스트를 토큰화하는 함수
def tokenize_texts(texts, labels):
    encodings = tokenizer(
        list(texts),  # 리스트 형태로 변환
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN, 
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"], tf.convert_to_tensor(labels, dtype=tf.int32)

In [18]:
# 데이터 분할 (8:2 비율로 train/val 나누기)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['conversation'], train_data['int_class'], test_size=0.2, random_state=42
)

In [19]:
# 데이터 토큰화
train_input_ids, train_attention_mask, train_labels = tokenize_texts(train_texts, train_labels)
val_input_ids, val_attention_mask, val_labels = tokenize_texts(val_texts, val_labels)

In [20]:
# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": train_input_ids, "attention_mask": train_attention_mask}, train_labels)
).shuffle(len(train_texts)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": val_input_ids, "attention_mask": val_attention_mask}, val_labels)
).batch(BATCH_SIZE)

In [21]:
# 데이터 변환 (train_data['conversation'], train_data['int_class'] 사용)
input_ids, attention_mask, label_tensor = tokenize_texts(train_data['conversation'], train_data['int_class'])

# TensorFlow Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": input_ids, "attention_mask": attention_mask}, label_tensor)
)

# 배치 및 섞기
dataset = dataset.shuffle(len(train_data)).batch(BATCH_SIZE)

In [22]:
attention_mask

<tf.Tensor: shape=(4266, 388), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [23]:
input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# BERT 모델에 입력
bert_output = model(input_ids, attention_mask=attention_mask)[1]
x = Dropout(0.1)(bert_output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
output = Dense(NUM_LABELS, activation="softmax")(x)  # 다중 클래스 분류

# 모델 정의
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# 모델 구조 확인
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 388)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 388)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108918528   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 768)          0           tf_bert_model[0][1]          

In [24]:
# EarlyStopping 콜백 설정 (patience=2)
early_stopping = EarlyStopping(
    monitor="val_loss",  # 검증 데이터 손실 기준
    patience=2,  # 2 epoch 동안 개선되지 않으면 종료
    restore_best_weights=True  # 가장 성능이 좋았던 가중치 복원
)

In [25]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping]  # 조기 종료 콜백 추가
)

Epoch 1/5


/opt/conda/lib/python3.9/site-packages/keras/backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


427/427 [==============================] - 342s 766ms/step - loss: 0.5107 - accuracy: 0.8200 - val_loss: 0.3533 - val_accuracy: 0.8888
Epoch 2/5
427/427 [==============================] - 324s 760ms/step - loss: 0.1630 - accuracy: 0.9513 - val_loss: 0.2909 - val_accuracy: 0.9075
Epoch 3/5
427/427 [==============================] - 325s 761ms/step - loss: 0.0713 - accuracy: 0.9763 - val_loss: 0.3238 - val_accuracy: 0.9145
Epoch 4/5
427/427 [==============================] - 325s 761ms/step - loss: 0.0457 - accuracy: 0.9842 - val_loss: 0.4231 - val_accuracy: 0.9110


In [31]:
from sklearn.metrics import f1_score

In [32]:
# 검증 데이터 예측
val_predictions = model.predict({"input_ids": val_input_ids, "attention_mask": val_attention_mask})
predicted_classes = np.argmax(val_predictions, axis=1)  # 확률값을 가장 높은 클래스 인덱스로 변환

# F1-score 계산
f1 = f1_score(val_labels, predicted_classes, average="weighted")

print(f"검증 데이터 F1-score: {f1:.4f}")

검증 데이터 F1-score: 0.9072


In [26]:
def tokenize_test_texts(texts):
    encodings = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"]

In [27]:
test_input_ids, test_attention_mask = tokenize_test_texts(test_data['text'])

In [28]:
# 모델 예측 수행
predictions = model.predict({"input_ids": test_input_ids, "attention_mask": test_attention_mask})

# 확률값을 가장 높은 클래스 인덱스로 변환
predicted_classes = np.argmax(predictions, axis=1)

# 결과를 데이터프레임에 추가
test_data['predicted_class'] = predicted_classes

In [29]:
# 예측된 결과 출력
print(test_data[['text', 'predicted_class']])

                                                  text  predicted_class
0    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...                1
1    우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...                2
2    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...                2
3    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...                4
4    아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...                3
..                                                 ...              ...
495  미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...                2
496  교수님 제 논문에 제 이름이 없나요?  아 무슨 논문말이야?  지난 번 냈던 논문이...                2
497  야 너  네 저요? 그래 너 왜요 돈좀 줘봐  돈 없어요 돈이 왜 없어 지갑은 폼이...                1
498  야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...                2
499  엄마 저 그 돈 안해주시면 정말 큰일나요.  이유도 말하지 않고. 몇번째니 경민아....                0

[500 rows x 2 columns]


In [33]:
test_data

,idx,text,predicted_class
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,1
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,2
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,2
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,4
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,3
...,...,...,...
495,t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,2
496,t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,2
497,t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,1
498,t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,2


In [35]:
# CSV로 저장
test_data[['idx', 'predicted_class']].to_csv("submission_kcvert.csv", index=False)